# Convolutional Neural Network Applications

* Resnet
* Inception
* 

# Image Segmentation

# Object Detection

# Image Captioning

* Show attend and tell

* Transposed Convolution
* FCN
* Unet
* PSP

* YOLO
* SSD